In [ ]:
import torch
import torchvision
import torch
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import random
from types import ModuleType

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
c = ModuleType("contstants")
# ------------------------------------- Constants -------------------------------------
c.Classes = 0
c.Training_Data = 1
c.Testing_Data = 2
c.Model = 3
c.Client_Accuracies = 4
c.Client_Performance_Scores = 5
c.Current_Performance_Score = 6
c.Client_Age = 7

NUM_CLIENTS = 10

"""
+---------------------------------------------------------------------------------+
|    ____ ___ _____ _    ____       _  ___    ____        _                 _     |
|   / ___|_ _|  ___/ \  |  _ \     / |/ _ \  |  _ \  __ _| |_ __ _ ___  ___| |_   |
|  | |    | || |_ / _ \ | |_) |____| | | | | | | | |/ _` | __/ _` / __|/ _ \ __|  |
|  | |___ | ||  _/ ___ \|  _ <_____| | |_| | | |_| | (_| | || (_| \__ \  __/ |_   |
|   \____|___|_|/_/   \_\_| \_\    |_|\___/  |____/ \__,_|\__\__,_|___/\___|\__|  |
+---------------------------------------------------------------------------------+
----------------------------------------------------------------------------------
+---------------------------------------------+
|   _   _                 _____ ___________   |
|  | \ | |               |_   _|_   _|  _  \  |
|  |  \| | ___  _ __ ______| |   | | | | | |  |
|  | . ` |/ _ \| '_ \______| |   | | | | | |  |
|  | |\  | (_) | | | |    _| |_ _| |_| |/ /   |
|  \_| \_/\___/|_| |_|    \___/ \___/|___/    |
+---------------------------------------------+
"""
def non_iid_distribution_CIFAR(NUM_CLIENTS):
    # Define the classes to include in each data loader
    classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    model = CIFAR10_CNN()
    model.to(device)
    class_indices = []
    client_map = dict()
    for client in range(NUM_CLIENTS):
        # Randomly select 2 classes for the client
        class_indices.append(np.random.choice(classes, 2, replace=False))
        
        # Print the selected classes
        print(f'Client {client}: {class_indices[client]}')

    # Define the transform to apply to the dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load the CIFAR-10 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

    # Create a list to store the data loaders
    train_data_loaders = []
    test_data_loaders = []
    # Create a data loader for each class combination
    for indices in class_indices:
        # Filter the dataset to include only the selected classes
        filtered_trainset = torch.utils.data.Subset(trainset, [i for i in range(len(trainset)) if trainset.targets[i] in indices])
        cli_train, cli_test = train_test_split(filtered_trainset, test_size=0.2)
        cli_train_loader = torch.utils.data.DataLoader(cli_train, batch_size=32, shuffle=True, num_workers=2)
        cli_test_loader = torch.utils.data.DataLoader(cli_test, batch_size=32, shuffle=True, num_workers=2)
        train_data_loaders.append(cli_train_loader)
        test_data_loaders.append(cli_test_loader)

    for client in range(NUM_CLIENTS):
        random_score = random.random()
        client_map.update({client: [class_indices[client], train_data_loaders[client], test_data_loaders[client], model, [0.0], [random_score], random_score, 0]})
    
    return client_map

""" 
+-----------------------+
|   _____ ___________   |
|  |_   _|_   _|  _  \  |
|    | |   | | | | | |  |
|    | |   | | | | | |  |
|   _| |_ _| |_| |/ /   |
|   \___/ \___/|___/    |
+-----------------------+

"""
def iid_distribution_CIFAR(NUM_CLIENTS):
    placeholder = [-1, -1]
    # Define the transform to apply to the dataset
    client_map = dict()
    model = CIFAR10_CNN()
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load the CIFAR-10 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

    # Split the dataset into 10 equal parts
    trainsets = torch.utils.data.random_split(trainset, [int(len(trainset) / NUM_CLIENTS) for _ in range(NUM_CLIENTS)])

    # Create a list to store the data loaders
    train_data_loaders = []
    test_data_loaders = []

    for client in range(NUM_CLIENTS):
        cli_train, cli_test = train_test_split(trainsets[client], test_size=0.2)
        cli_train_loader = torch.utils.data.DataLoader(cli_train, batch_size=32, shuffle=True, num_workers=2)
        cli_test_loader = torch.utils.data.DataLoader(cli_test, batch_size=32, shuffle=True, num_workers=2)
        train_data_loaders.append(cli_train_loader)
        test_data_loaders.append(cli_test_loader)
    
    for client in range(NUM_CLIENTS):
        random_score = random.random()
        client_map.update({client: [placeholder, train_data_loaders[client], test_data_loaders[client], model, [0.0], [random_score], random_score, 0]})
    
    return client_map

""""
+---------------------------------------------------------------------+
|   __  __ _   _ ___ ____ _____   ____        _                 _     |
|  |  \/  | \ | |_ _/ ___|_   _| |  _ \  __ _| |_ __ _ ___  ___| |_   |
|  | |\/| |  \| || |\___ \ | |   | | | |/ _` | __/ _` / __|/ _ \ __|  |
|  | |  | | |\  || | ___) || |   | |_| | (_| | || (_| \__ \  __/ |_   |
|  |_|  |_|_| \_|___|____/ |_|   |____/ \__,_|\__\__,_|___/\___|\__|  |
+---------------------------------------------------------------------+

+---------------------------------------------+
|   _   _                 _____ ___________   |
|  | \ | |               |_   _|_   _|  _  \  |
|  |  \| | ___  _ __ ______| |   | | | | | |  |
|  | . ` |/ _ \| '_ \______| |   | | | | | |  |
|  | |\  | (_) | | | |    _| |_ _| |_| |/ /   |
|  \_| \_/\___/|_| |_|    \___/ \___/|___/    |
+---------------------------------------------+
"""
def non_iid_distribution_MNIST(NUM_CLIENTS):
    # Define the classes to include in each data loader
    classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    class_indices = []
    client_map = dict()
    model = MNIST_MLP()
    for client in range(NUM_CLIENTS):
        # Randomly select 2 classes for the client
        class_indices.append(np.random.choice(classes, 2, replace=False))
        
        # Print the selected classes
        print(f'Client {client}: {class_indices[client]}')

    # Define the transform to apply to the dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load the MNIST dataset
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

    # Create a list to store the data loaders
    train_data_loaders = []
    test_data_loaders = []
    # Create a data loader for each class combination
    for indices in class_indices:
        # Filter the dataset to include only the selected classes
        filtered_trainset = torch.utils.data.Subset(trainset, [i for i in range(len(trainset)) if trainset.targets[i] in indices])
        cli_train, cli_test = train_test_split(filtered_trainset, test_size=0.2)
        cli_train_loader = torch.utils.data.DataLoader(cli_train, batch_size=32, shuffle=True, num_workers=2)
        cli_test_loader = torch.utils.data.DataLoader(cli_test, batch_size=32, shuffle=True, num_workers=2)
        train_data_loaders.append(cli_train_loader)
        test_data_loaders.append(cli_test_loader)

    for client in range(NUM_CLIENTS):
        random_score = random.random()
        client_map.update({client: [class_indices[client], train_data_loaders[client], test_data_loaders[client], model, [0.0], [random_score], random_score, 0]})
    
    return client_map

"""
+-----------------------+
|   _____ ___________   |
|  |_   _|_   _|  _  \  |
|    | |   | | | | | |  |
|    | |   | | | | | |  |
|   _| |_ _| |_| |/ /   |
|   \___/ \___/|___/    |
+-----------------------+
"""
def iid_distribution_MNIST(NUM_CLIENTS):
    placeholder = -1
    # Define the transform to apply to the dataset
    client_map = dict()
    model = MNIST_MLP()
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Load the MNIST dataset
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

    # Split the dataset into 10 equal parts
    trainsets = torch.utils.data.random_split(trainset, [int(len(trainset) / NUM_CLIENTS) for _ in range(NUM_CLIENTS)])

    # Create a list to store the data loaders
    train_data_loaders = []
    test_data_loaders = []

    for client in range(NUM_CLIENTS):
        cli_train, cli_test = train_test_split(trainsets[client], test_size=0.2)
        cli_train_loader = torch.utils.data.DataLoader(cli_train, batch_size=32, shuffle=True, num_workers=2)
        cli_test_loader = torch.utils.data.DataLoader(cli_test, batch_size=32, shuffle=True, num_workers=2)
        train_data_loaders.append(cli_train_loader)
        test_data_loaders.append(cli_test_loader)
    
    for client in range(NUM_CLIENTS):
        random_score = random.random()
        client_map.update({client: [placeholder, train_data_loaders[client], test_data_loaders[client], model, [0.0], [random_score], random_score, 0]})
    
    return client_map


"""
+---------------------------------------------------------------------------+
|   _   _                      _   _   _      _                      _      |
|  | \ | |                    | | | \ | |    | |                    | |     |
|  |  \| | ___ _   _ _ __ __ _| | |  \| | ___| |___      _____  _ __| | __  |
|  | . ` |/ _ \ | | | '__/ _` | | | . ` |/ _ \ __\ \ /\ / / _ \| '__| |/ /  |
|  | |\  |  __/ |_| | | | (_| | | | |\  |  __/ |_ \ V  V / (_) | |  |   <   |
|  \_| \_/\___|\__,_|_|  \__,_|_| \_| \_/\___|\__| \_/\_/ \___/|_|  |_|\_\  |
+---------------------------------------------------------------------------+
"""
# ------------------------------------- MLP MNIST -------------------------------------
class MNIST_MLP(nn.Module):
    def __init__(self):
        super(MNIST_MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
# ------------------------------------- CNN CIFAR-10 -------------------------------------
class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x


In [ ]:
def initialize_clients(NUM_CLIENTS, dataset, distribution):
    if dataset == 'MNIST':
        if distribution == 'iid':
            client_map = iid_distribution_MNIST(NUM_CLIENTS)
        else:
            client_map = non_iid_distribution_MNIST(NUM_CLIENTS)
    else:
        if distribution == 'iid':
            client_map = iid_distribution_CIFAR(NUM_CLIENTS)
        else:
            client_map = non_iid_distribution_CIFAR(NUM_CLIENTS)
    
    return client_map

#### The client map for the non-iid data distribution contains the following information associated with that client
##### -Key: Client_Number | Values: Two_Classes, Training_Data, Testing_Data, Nueral_Network, Client Accuracies, Client Performance Scores, Current Performance Score

#### The client map for the iid data distribution contains the following information associated with that client
#####  -Key: Client_Number | Values: Placeholder, Training_Data, Testing_Data, Nueral_Network, Client Accuracies, Client Performance Scores,  Current Performance Score


#### The client map for the iid data distribution contains the following information associated with that client
#####  -Key: Client_Number | Values: Placeholder, Training_Data, Testing_Data, Nueral_Network, Client Accuracies, Client Performance Scores, Current Performance Score

In [ ]:
import random
client_performance_score = dict()
client_accuracies = dict()
client_actual_score = dict()
rounds = 10
threshold = 0.8
for client in range(NUM_CLIENTS):
    random_score = random.random()
    client_performance_score.update({client: [random_score]})
    client_accuracies.update({client: [0.0]})
    client_actual_score.update({client: 0.0}) 

for round in range(rounds):
    for client in  range(NUM_CLIENTS):
        random_num = random.random()*2
        client_accuracies[client].append(random_num)
        sel_client_acc = client_accuracies[client]
        mean_score = sum(sel_client_acc) / len(sel_client_acc)
        varience = sum([((x - mean_score) ** 2) for x in sel_client_acc]) / (len(sel_client_acc))
        client_zero_score = client_actual_score[1]
        if varience > threshold:
            updated_score = client_performance_score[client][round] + 1.0
            client_performance_score[client].append(updated_score)
        else:
            updated_score = client_performance_score[client][round]
            client_performance_score[client].append(updated_score)
        mean_performance_score = sum(client_performance_score[client]) / len(client_performance_score[client])
        client_performance_score[client][round+1] = mean_performance_score
        client_actual_score.update({client: [mean_performance_score]})


In [ ]:
client_ages = dict()
rounds = 100
client_arr = []
selected_clients = []
clients_per_rnd = 10
probability_of_comm = 0.8
for i in range(NUM_CLIENTS):
    client_arr.append(i)

client_ages.update({9: 1})
for client in range(NUM_CLIENTS-1):
    client_ages.update({client: 0})

for round in range(rounds):
    sorted_clients = sorted(client_ages.items(), key=lambda x: x[1], reverse=True)
    selected_clients = [x[0] for x in sorted_clients[:clients_per_rnd]]
    for client in range(NUM_CLIENTS):
        if client in selected_clients:
            client_ages[client] = 0
        else:
            client_ages[client] = client_ages[client]**2 + 1

In [ ]:
def train_and_test_client(client_map, selected_clients, local_epochs, learning_rate):
    for client in selected_clients:
        model = client_map[client][c.Model]
        model.to(device)
        train_loader = client_map[client][c.Training_Data]
        test_loader = client_map[client][c.Testing_Data]
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
        for epoch in range(local_epochs):
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                if i % 2000 == 1999:
                    print(f'Client: {client} Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 2000}')
                    running_loss = 0.0
            
        client_map[client][c.Model] = model
    return client_map